In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import time

# 用pd.read_html抓取資料表格

In [7]:
station_info = pd.read_csv('station_info.csv')

FileNotFoundError: File b'station_info.csv' does not exist

In [ ]:
station_info.drop('Unnamed: 0',axis = 1,inplace=True)   #axis:去行（0）或列（1）

In [ ]:
station_info

In [ ]:
stationIndexList = station_info.columns

# [降雨量, 溫度] = getClimate(site,date,mode)
## 需先執行
- station_info = pd.read_csv('station_info.csv')
- station_info.drop('Unnamed: 0',axis = 1,inplace=True)
- stationIndexList = station_info.columns

## site : 地區 (ex:臺北市)
## date : 日期 (ex:2018-06-05)
## mode=1 : 較慢但較準，採取人工測站與無人測站，且加入delay防鎖IP
## mode=2 : 較快但不準，只採取人工測站，未加入delay

In [ ]:
def getClimate(site,date,mode):
    
    precpAvgList = list()
    tempAvgList = list()
    
    #台-->臺
    if site[0] == '台':
        site = '臺'+site[1:] #字串無法直接指定, ex:site[k] = '臺'
            
    if mode == 1: 
        for i in range(len(stationIndexList)):       
            
            #此地區每個測站
            if station_info[stationIndexList[i]][2] == site:
                
                #需要delay才不會被擋
                time.sleep(0.11) 
                
                #去指定網址爬取
#                 print(station_info[stationIndexList[i]][4] + date) #網址
                temp=pd.read_html(station_info[stationIndexList[i]][4] + date)
                
                 #此測站24小時的資料
                for j in range(len(temp[1][10][2:])):
                    
                    #清空
                    precpList = list()
                    tempList = list() 
                    
                    #讀取，若是X則跳過
                    try:
                        precpList.append(np.float(temp[1][10][j+2]))
                        tempList.append(np.float(temp[1][3][j+2]))
                    except:
                        continue
                        
                    #求得此測站平均
                    precpAvg = np.average(precpList)
                    tempAvg = np.average(tempList)
                    
                #此地區所有測站值
                if precpAvg == precpAvg:  #確認是否為nan
                    precpAvgList.append(precpAvg)
                if tempAvg == tempAvg:
                    tempAvgList.append(tempAvg)
                    
        #算出所有測站平均回傳 
#         print(precpAvgList)
#         print(tempAvgList)
        
        return np.average(precpAvgList), np.average(tempAvgList)
    
    else:
        for i in range(len(stationIndexList)):
            
            #此地區每個人工測站
            if (station_info[stationIndexList[i]][2] == site) and stationIndexList[i].isdigit():
                
                #去指定網址爬取
#                 print(station_info[stationIndexList[i]][4] + date) #網址
                temp=pd.read_html(station_info[stationIndexList[i]][4] + date)
    
                 #此測站24小時的資料
                for j in range(len(temp[1][10][2:])):
                    #清空
                    precpList = list()
                    tempList = list() 
                    
                    #讀取，若是X則跳過
                    try:
                        precpList.append(np.float(temp[1][10][j+2]))
                        tempList.append(np.float(temp[1][3][j+2]))
                    except:
                        continue
                        
                    #求得此測站平均
                    precpAvg = np.average(precpList)
                    tempAvg = np.average(tempList)
                
                #此地區所有測站值
                if precpAvg == precpAvg:  #確認是否為nan
                    precpAvgList.append(precpAvg)
                if tempAvg == tempAvg:
                    tempAvgList.append(tempAvg)
#             else:
#                 break
        #算出人工測站平均回傳    
#         print(precpAvgList)
#         print(tempAvgList)
        return np.average(precpAvgList), np.average(tempAvgList)

## example

In [ ]:
getClimate('桃園市','2018-03-06',1)

In [ ]:
getClimate('中壢區','2018-03-06',2)

- 應該沒差多少吧